<a href="https://colab.research.google.com/github/laurelkeys/machine-learning/blob/master/assignment-4/Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformer implementation
Replicating the code (and explanations) from [Transformer model for language understanding](https://www.tensorflow.org/tutorials/text/transformer) TensorFlow tutorial, to build upon it.

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import time
import numpy as np
import matplotlib.pyplot as plt

In [0]:
import warnings

In [0]:
import functools

SPACES = 0
def indent_prints(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        global SPACES
        SPACES += 2
        try:
            ret = func(*args, **kwargs)
        except:
            ret = func(*args)
        SPACES -= 2
        return ret
    return wrapper

## Positional encoding

Since this model doesn't contain any recurrence or convolution, positional encoding is added to give the model some information about the relative position of the words in the sentence. 

The positional encoding vector is added to the embedding vector. Embeddings represent a token in a d-dimensional space where tokens with similar meaning will be closer to each other. But the embeddings do not encode the relative position of words in a sentence. So after adding the positional encoding, words will be closer to each other based on the *similarity of their meaning and their position in the sentence*, in the d-dimensional space.

See the notebook on [positional encoding](https://github.com/tensorflow/examples/blob/master/community/en/position_encoding.ipynb) to learn more about it. The formula for calculating the positional encoding is as follows:

${PE_{(pos,\,2i)} = sin(pos / 10000^{2i / d_{model}})}\,,\;\;\;{PE_{(pos,\,2i+1)} = cos(pos / 10000^{2i / d_{model}})} $

In [0]:
@indent_prints
def get_angles(pos, i, d_model):
    print(" "*SPACES, "[get_angles] pos", pos.shape)
    print(" "*SPACES, "[get_angles] i", i.shape)
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

In [0]:
@indent_prints
def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                            np.arange(d_model)[np.newaxis, :],
                            d_model)
    print(" "*SPACES, "[positional_encoding] angle_rads", angle_rads.shape)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]
    return tf.cast(pos_encoding, dtype=tf.float32)

## Masking


In [0]:
#    atari image    ==>   resized image   ==MobileNetV2==>  feature vector  ==GlobalAveragePooling2D==>   token
# (bs, 210, 160, 3) ==> (bs, 160, 160, 3) ==MobileNetV2==> (bs, 5, 5, 1280) ==GlobalAveragePooling2D==> (bs, 1280)

# (bs, input_seq_len, d_model) ==Transformer==> ...
# bs            : batch_size
# input_seq_len : 1000 for training (could be less if we want to use it as a policy network, predicting actions on-the-fly)
# d_model       : 1280 (MobileNetV2 output with GlobalAveragePooling2D)

Mask all the pad tokens in the batch of sequence. It ensures that the model does not treat padding as the input. The mask indicates where pad value `PAD_VALUE` is present: it outputs a `1` at those locations, and a `0` otherwise.

In [0]:
PAD_VALUE = 0 # FIXME

def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, PAD_VALUE), tf.float32)

    # add extra dimensions to add the padding to the attention logits
    return seq[:, tf.newaxis, tf.newaxis, :] # (batch_size, 1, 1, seq_len)

The look-ahead mask is used to mask the future tokens in a sequence. In other words, the mask indicates which entries should not be used.

This means that to predict the third token, only the first and second tokens will be used. Similarly to predict the fourth token, only the first, second and the third tokens will be used and so on.

In [0]:
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask # (seq_len, seq_len)

In [246]:
create_look_ahead_mask(size=4)

<tf.Tensor: id=372780, shape=(4, 4), dtype=float32, numpy=
array([[0., 1., 1., 1.],
       [0., 0., 1., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.]], dtype=float32)>

## Scaled dot product attention
The attention function used by the transformer takes three inputs: Q (query), K (key), V (value). The equation used to calculate the attention weights is: ${Attention(Q, K, V) = softmax_k(\frac{QK^T}{\sqrt{d_k}}) V} $

The mask is multiplied by $-10^9$ and summed with the scaled matrix multiplication of Q and K, and is applied immediately before a softmax. The goal is to zero out these cells, as large negative inputs to softmax are near zero in the output.

<img src="https://www.tensorflow.org/images/tutorials/transformer/scaled_attention.png" width="300" alt="scaled_dot_product_attention">

In [0]:
@indent_prints
def scaled_dot_product_attention(q, k, v, mask=None):
    ''' Calculate Attention(q, k, v) and the attention weights.
        Args:
          q: query shape == (..., seq_len_q, depth)
          k: key shape   == (..., seq_len_k, depth)
          v: value shape == (..., seq_len_v, depth_v)
          mask: float tensor with shape broadcastable to (..., seq_len_q, seq_len_k)
        Returns:
          output, attention_weights '''
    print(" "*SPACES, "[scaled_dot_product_attention]")
    print(" "*SPACES, "- q", q.shape)
    print(" "*SPACES, "- k", k.shape)
    print(" "*SPACES, "- v", v.shape)
    
    # assert q.shape[:-2] == k.shape[:-2] == v.shape[:-2], "q, k, v must have matching leading dimensions"
    # assert k.shape[-2] == v.shape[-2], "k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v"

    matmul_qk = tf.matmul(q, k, transpose_b=True) # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    d_k = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(d_k)

    # add the (padding or look ahead) mask to the scaled tensor
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k) so that the scores add up to 1
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1) # (..., seq_len_q, seq_len_k)
    
    output = tf.matmul(attention_weights, v) # (..., seq_len_q, depth_v)
    
    return output, attention_weights

As the softmax normalization is done on K, its values decide the amount of importance given to Q.

The output represents the multiplication of the attention weights and the V (value) vector. This ensures that the tokens you want to focus on are kept as-is and the irrelevant tokens are flushed out.

### Example

In [0]:
def print_out(q, k, v):
    temp_out, temp_attn = scaled_dot_product_attention(q, k, v, None)
    print('Attention weights are:', temp_attn)
    print('Output is:            ', temp_out)
np.set_printoptions(suppress=True)

In [0]:
# def print_out_shape(q, k, v):
#     temp_out, temp_attn = scaled_dot_product_attention(q, k, v, None)
#     print('Attention weights are:', temp_attn.shape)
#     print('Output is:            ', temp_out.shape)

# print_out_shape(
#     q=tf.random.uniform((64, 10, 24 // 8)), # (batch_size, seq_len, d_model_q)
#     k=tf.random.uniform((64, 10, 1280 // 8)), # (batch_size, seq_len, d_model)
#     v=tf.random.uniform((64, 10, 1280 // 8))) # (batch_size, seq_len, d_model)

In [0]:
temp_k = tf.constant([[10,0,0], [0,10,0], [0,0,10], [0,0,10]], dtype=tf.float32) # (4, 3)

temp_v = tf.constant([[   1,0], [  10,0], [ 100,5], [1000,6]], dtype=tf.float32) # (4, 2)

In [251]:
# This query aligns with the second key, so the second value is returned
temp_q = tf.constant([[0, 10, 0]], dtype=tf.float32) # (1, 3)
print_out(temp_q, temp_k, temp_v)

   [scaled_dot_product_attention]
   - q (1, 3)
   - k (4, 3)
   - v (4, 2)
Attention weights are: tf.Tensor([[0. 1. 0. 0.]], shape=(1, 4), dtype=float32)
Output is:             tf.Tensor([[10.  0.]], shape=(1, 2), dtype=float32)


In [252]:
# This query aligns with a repeated key (third and fourth), so all associated values get averaged
temp_q = tf.constant([[0, 0, 10]], dtype=tf.float32) # (1, 3)
print_out(temp_q, temp_k, temp_v)

   [scaled_dot_product_attention]
   - q (1, 3)
   - k (4, 3)
   - v (4, 2)
Attention weights are: tf.Tensor([[0.  0.  0.5 0.5]], shape=(1, 4), dtype=float32)
Output is:             tf.Tensor([[550.    5.5]], shape=(1, 2), dtype=float32)


In [253]:
# This query aligns equally with the first and second key, so their values get averaged
temp_q = tf.constant([[10, 10, 0]], dtype=tf.float32) # (1, 3)
print_out(temp_q, temp_k, temp_v)

   [scaled_dot_product_attention]
   - q (1, 3)
   - k (4, 3)
   - v (4, 2)
Attention weights are: tf.Tensor([[0.5 0.5 0.  0. ]], shape=(1, 4), dtype=float32)
Output is:             tf.Tensor([[5.5 0. ]], shape=(1, 2), dtype=float32)


In [254]:
# Pass all the queries together
temp_q = tf.constant([[0, 0, 10], [0, 10, 0], [10, 10, 0]], dtype=tf.float32) # (3, 3)
print_out(temp_q, temp_k, temp_v)

   [scaled_dot_product_attention]
   - q (3, 3)
   - k (4, 3)
   - v (4, 2)
Attention weights are: tf.Tensor(
[[0.  0.  0.5 0.5]
 [0.  1.  0.  0. ]
 [0.5 0.5 0.  0. ]], shape=(3, 4), dtype=float32)
Output is:             tf.Tensor(
[[550.    5.5]
 [ 10.    0. ]
 [  5.5   0. ]], shape=(3, 2), dtype=float32)


## Multi-head attention
Multi-head attention consists of four parts:
* Linear layers and split into heads
* Scaled dot-product attention
* Concatenation of heads
* Final linear layer

<img src="https://www.tensorflow.org/images/tutorials/transformer/multi_head_attention.png" width="300" alt="multi-head attention">

Each multi-head attention block gets three inputs; Q (query), K (key), and V (value). These are put through linear (Dense) layers and split up into multiple heads. 

The `scaled_dot_product_attention` defined above is applied to each head (broadcasted for efficiency). An appropriate mask must be used in the attention step.  The attention output for each head is then concatenated (using `tf.transpose`, and `tf.reshape`) and put through a final `Dense` layer.

Instead of one single attention head, Q, K, and V are split into multiple heads because it allows the model to jointly attend to information at different positions from different representational spaces. After the split each head has a reduced dimensionality, so the total computation cost is the same as a single head attention with full dimensionality.

In [0]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.depth = d_model // num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        # NOTE I think this is d_model_q (and not d_model)
        self.dense = tf.keras.layers.Dense(d_model)
    
    @indent_prints
    def split_heads(self, x, batch_size):
        ''' Split the last dimension into (num_heads, depth).
            Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth) '''
        print(" "*SPACES, "[split_heads]")
        print(" "*SPACES,"- batch_size", batch_size)
        print(" "*SPACES,"- x", x.shape)
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        print(" "*SPACES,"- x (reshape)", x.shape)
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    @indent_prints
    def call(self, v, k, q, mask=None):
        print(" "*SPACES, "[MultiHeadAttention.call]")
        print(" "*SPACES,"- v", v.shape)
        print(" "*SPACES,"- k", k.shape)
        print(" "*SPACES,"- q", q.shape)
        batch_size = tf.shape(q)[0]
        
        q = self.wq(q) # (batch_size, seq_len, d_model)
        k = self.wk(k) # (batch_size, seq_len, d_model)
        v = self.wv(v) # (batch_size, seq_len, d_model)

        # obs.: seq_len_k == seq_len_v
        q = self.split_heads(q, batch_size)   # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)   # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)   # (batch_size, num_heads, seq_len_v, depth)

        # attention.shape         == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        attention, attention_weights = scaled_dot_product_attention(q, k, v, mask)

        print(" "*SPACES,"- scaled_dot_product_attention", attention.shape, attention_weights.shape)

        attention = tf.transpose(attention, perm=[0, 2, 1, 3]) # (batch_size, seq_len_q, num_heads, depth)
        # NOTE I think this is d_model (and not d_model)
        print(" "*SPACES,"- attention (transposed)", attention.shape)
        concat_attention = tf.reshape(attention, (batch_size, -1, self.d_model)) # (batch_size, seq_len_q, d_model)
        print(" "*SPACES,"- concat_attention", concat_attention.shape)
        output = self.dense(concat_attention) # (batch_size, seq_len_q, d_model)
        print(" "*SPACES,"- output", concat_attention.shape)

        return output, attention_weights

In [256]:
sample_multiheadattention_layer = MultiHeadAttention(24, 8)

x_sample = tf.random.uniform((64, 100, 24))
sample_multiheadattention_layer_output, _ = sample_multiheadattention_layer(
    x_sample, x_sample, x_sample, None)

sample_multiheadattention_layer_output.shape # (batch_size, input_seq_len, d_model)

   [MultiHeadAttention.call]
   - v (64, 100, 24)
   - k (64, 100, 24)
   - q (64, 100, 24)
     [split_heads]
     - batch_size tf.Tensor(64, shape=(), dtype=int32)
     - x (64, 100, 24)
     - x (reshape) (64, 100, 8, 3)
     [split_heads]
     - batch_size tf.Tensor(64, shape=(), dtype=int32)
     - x (64, 100, 24)
     - x (reshape) (64, 100, 8, 3)
     [split_heads]
     - batch_size tf.Tensor(64, shape=(), dtype=int32)
     - x (64, 100, 24)
     - x (reshape) (64, 100, 8, 3)
     [scaled_dot_product_attention]
     - q (64, 8, 100, 3)
     - k (64, 8, 100, 3)
     - v (64, 8, 100, 3)
   - scaled_dot_product_attention (64, 8, 100, 3) (64, 8, 100, 100)
   - attention (transposed) (64, 100, 8, 3)
   - concat_attention (64, 100, 24)
   - output (64, 100, 24)


TensorShape([64, 100, 24])

In [257]:
sample_multiheadattention_layer = MultiHeadAttention(24, 8)

x_sample = tf.random.uniform((64, 100, 24))
y_sample = tf.random.uniform((64, 100, 1280))
sample_multiheadattention_layer_output, _ = sample_multiheadattention_layer(
    y_sample, y_sample, x_sample, None)

sample_multiheadattention_layer_output.shape # (batch_size, input_seq_len, d_model)

   [MultiHeadAttention.call]
   - v (64, 100, 1280)
   - k (64, 100, 1280)
   - q (64, 100, 24)
     [split_heads]
     - batch_size tf.Tensor(64, shape=(), dtype=int32)
     - x (64, 100, 24)
     - x (reshape) (64, 100, 8, 3)
     [split_heads]
     - batch_size tf.Tensor(64, shape=(), dtype=int32)
     - x (64, 100, 24)
     - x (reshape) (64, 100, 8, 3)
     [split_heads]
     - batch_size tf.Tensor(64, shape=(), dtype=int32)
     - x (64, 100, 24)
     - x (reshape) (64, 100, 8, 3)
     [scaled_dot_product_attention]
     - q (64, 8, 100, 3)
     - k (64, 8, 100, 3)
     - v (64, 8, 100, 3)
   - scaled_dot_product_attention (64, 8, 100, 3) (64, 8, 100, 100)
   - attention (transposed) (64, 100, 8, 3)
   - concat_attention (64, 100, 24)
   - output (64, 100, 24)


TensorShape([64, 100, 24])

## Point wise feed forward network
Point wise feed forward network consists of two fully-connected layers with a ReLU activation in between.

In [0]:
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'), # (batch_size, seq_len, dff)
        tf.keras.layers.Dense(d_model) # (batch_size, seq_len, d_model)
    ])

## Encoder and decoder
* The input sentence is passed through N encoder layers that generates an output for each token in the sequence.
* The decoder attends on the encoder's output and its own input (self-attention) to predict the next token.

<img src="https://www.tensorflow.org/images/tutorials/transformer/transformer.png" width="450" alt="transformer">

### Encoder layer
Each encoder layer consists of sublayers:
1. Multi-head attention 
2. Point wise feed forward networks. 

Each of these sublayers has a residual connection around it followed by a layer normalization. Residual connections help in avoiding the vanishing gradient problem in deep networks.

The output of each sublayer is `LayerNorm(x + Sublayer(x))`. The normalization is done on the `d_model` (last) axis. There are N encoder layers in the transformer.

In [0]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask):
        # x.shape == (batch_size, input_seq_len, d_model)
        attn_output, _ = self.mha(x, x, x, mask) # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output) # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1) # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output) # (batch_size, input_seq_len, d_model)
        
        return out2 # shape same as x

In [260]:
sample_encoder_layer = EncoderLayer(1280, 8, 256)

sample_encoder_layer_output = sample_encoder_layer(
    tf.random.uniform((32, 100, 1280)), False, None)

sample_encoder_layer_output.shape # (batch_size, input_seq_len, d_model)

   [MultiHeadAttention.call]
   - v (32, 100, 1280)
   - k (32, 100, 1280)
   - q (32, 100, 1280)
     [split_heads]
     - batch_size tf.Tensor(32, shape=(), dtype=int32)
     - x (32, 100, 1280)
     - x (reshape) (32, 100, 8, 160)
     [split_heads]
     - batch_size tf.Tensor(32, shape=(), dtype=int32)
     - x (32, 100, 1280)
     - x (reshape) (32, 100, 8, 160)
     [split_heads]
     - batch_size tf.Tensor(32, shape=(), dtype=int32)
     - x (32, 100, 1280)
     - x (reshape) (32, 100, 8, 160)
     [scaled_dot_product_attention]
     - q (32, 8, 100, 160)
     - k (32, 8, 100, 160)
     - v (32, 8, 100, 160)
   - scaled_dot_product_attention (32, 8, 100, 160) (32, 8, 100, 100)
   - attention (transposed) (32, 100, 8, 160)
   - concat_attention (32, 100, 1280)
   - output (32, 100, 1280)


TensorShape([32, 100, 1280])

### Decoder layer
Each decoder layer consists of sublayers:
1. Masked multi-head attention (with look ahead mask and padding mask)
2. Multi-head attention (with padding mask)  
   V (value) and K (key) receive the *encoder output* as inputs  
   Q (query) receives the *output from the masked multi-head attention sublayer*
3. Point wise feed forward networks

Each of these sublayers has a residual connection around it followed by a layer normalization. The output of each sublayer is `LayerNorm(x + Sublayer(x))`. The normalization is done on the `d_model` (last) axis.

There are N decoder layers in the transformer.

As Q receives the output from decoder's first attention block, and K receives the encoder output, the attention weights represent the importance given to the decoder's input based on the encoder's output. In other words, the decoder predicts the next word by looking at the encoder output and self-attending to its own output.

In [0]:
# d_model == 1280 (comes from the encoder, i.e. images' feature vectors)
# d_model_q == 18 (comes from the previous decoder output, i.e. actions)
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1, d_model_q=None):
        super(DecoderLayer, self).__init__()

        warnings.warn(f"d_model_q must be 18 as we have actions from 0 to 17 (not {d_model_q})")

        self.mha1 = MultiHeadAttention(d_model_q, num_heads)
        self.mha2 = MultiHeadAttention(d_model_q, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model_q, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)    
    
    @indent_prints
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        # x == (batch_size, target_seq_len, d_model_q)
        # enc_output.shape == (batch_size, input_seq_len, d_model)
        print(" "*SPACES, "[DecoderLayer.call]")
        print(" "*SPACES, "- x", x.shape)
        print(" "*SPACES, "- enc_output", enc_output.shape)

        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask) # (batch_size, target_seq_len, d_model_q)
        print(" "*SPACES, "- attn1", attn1.shape)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)
        print(" "*SPACES, "- out1", out1.shape)


        attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask) # (batch_size, target_seq_len, d_model_q)
        print(" "*SPACES, "- attn2", attn2.shape)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1) # (batch_size, target_seq_len, d_model_q)
        print(" "*SPACES, "- out2", out1.shape)

        ffn_output = self.ffn(out2) # (batch_size, target_seq_len, d_model_q)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2) # (batch_size, target_seq_len, d_model_q)
        print(" "*SPACES, "- out3", out1.shape)

        return out3, attn_weights_block1, attn_weights_block2

In [262]:
sample_decoder_layer = DecoderLayer(1280, 8, 256, d_model_q=24) # 24 since 18 isn't divisible by 8

sample_decoder_layer_output, _, _ = sample_decoder_layer(tf.random.uniform((32, 100, 24)), 
                                                         sample_encoder_layer_output, False, None, None)

sample_decoder_layer_output.shape # (batch_size, target_seq_len, d_model)

   [DecoderLayer.call]
   - x (32, 100, 24)
   - enc_output (32, 100, 1280)
     [MultiHeadAttention.call]
     - v (32, 100, 24)
     - k (32, 100, 24)
     - q (32, 100, 24)
       [split_heads]
       - batch_size tf.Tensor(32, shape=(), dtype=int32)
       - x (32, 100, 24)
       - x (reshape) (32, 100, 8, 3)
       [split_heads]
       - batch_size tf.Tensor(32, shape=(), dtype=int32)
       - x (32, 100, 24)
       - x (reshape) (32, 100, 8, 3)
       [split_heads]
       - batch_size tf.Tensor(32, shape=(), dtype=int32)
       - x (32, 100, 24)
       - x (reshape) (32, 100, 8, 3)
       [scaled_dot_product_attention]
       - q (32, 8, 100, 3)
       - k (32, 8, 100, 3)
       - v (32, 8, 100, 3)
     - scaled_dot_product_attention (32, 8, 100, 3) (32, 8, 100, 100)
     - attention (transposed) (32, 100, 8, 3)
     - concat_attention (32, 100, 24)
     - output (32, 100, 24)
   - attn1 (32, 100, 24)
   - out1 (32, 100, 24)
     [MultiHeadAttention.call]
     - v (32, 100, 1280

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: d_model_q must be 18 as we have actions from 0 to 17 (not 24)
  """


TensorShape([32, 100, 24])

### Encoder
The `Encoder` consists of:
1. Input Embedding
2. Positional Encoding
3. N encoder layers

The input is put through an embedding which is summed with the positional encoding. The output of this summation is the input to the encoder layers. The output of the encoder is the input to the decoder.

In [0]:
# NOTE replacing Embedding inside the Encoder with MobileNetV2 + GlobalAveragePooling2D
# The output should have shape == (batch_size, input_seq_len, d_model), where d_model will be 1280 as we're using MobileNetV2
class FeatureExtractor(tf.keras.layers.Layer):
    def __init__(self):
        super(FeatureExtractor, self).__init__()

        self.mobile_net_v2 = tf.keras.applications.MobileNetV2(input_shape=(160, 160, 3), 
                                                               include_top=False, 
                                                               weights='imagenet')
        self.mobile_net_v2.trainable = False
        
        self.global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
        
    @indent_prints
    def call(self, x):
        # reshaping (batch_size, input_seq_len, 160, 160, 3) to (batch_size * input_seq_len, 160, 160, 3)
        print(" "*SPACES, "[FeatureExtractor.call]")
        print(" "*SPACES, "- x", x.shape)
        x_shape = tf.shape(x)
        x = tf.reshape(x, (-1, x_shape[2], x_shape[3], x_shape[4]))
        print(" "*SPACES, "- x (reshaped)", x.shape)
        x = self.mobile_net_v2(x) # (batch_size * input_seq_len, 5, 5, 1280)
        print(" "*SPACES, "- x (mobile_net_v2)", x.shape)

        x = self.global_average_layer(x) # (batch_size * input_seq_len, 1280)
        print(" "*SPACES, "- x (global_average_layer)", x.shape)
        x = tf.reshape(x, (x_shape[0], x_shape[1], -1))
        print(" "*SPACES, "- x (reshaped)", x.shape)
        return x # (batch_size, input_seq_len, d_model), with d_model == 1280

In [264]:
sample_feature_extractor = FeatureExtractor()

temp_input = tf.random.uniform((2, 10, 160, 160, 3), dtype=tf.float32, minval=-1, maxval=1)

sample_feature_extractor_output = sample_feature_extractor(temp_input)

print (sample_feature_extractor_output.shape) # (batch_size, input_seq_len, d_model)

   [FeatureExtractor.call]
   - x (2, 10, 160, 160, 3)
   - x (reshaped) (20, 160, 160, 3)
   - x (mobile_net_v2) (20, 5, 5, 1280)
   - x (global_average_layer) (20, 1280)
   - x (reshaped) (2, 10, 1280)
(2, 10, 1280)


In [0]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, 
                 maximum_position_encoding, rate=0.1):
        super(Encoder, self).__init__()

        assert d_model == 1280, "d_model must be 1280 as we're using MobileNetV2"
        warnings.warn(f"input_vocab_size is no longer used (its value of {input_vocab_size} will be ignored)")

        self.d_model = d_model
        self.num_layers = num_layers

        # NOTE replacing Embedding with MobileNetV2 + GlobalAveragePooling2D
        # > self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.feature_extractor = FeatureExtractor()
        self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)
        
    def call(self, x, training, mask=None):
        seq_len = tf.shape(x)[1]

        # adding embedding and position encoding        
        # NOTE replacing Embedding with MobileNetV2 + GlobalAveragePooling2D
        # > x = self.embedding(x) # (batch_size, input_seq_len, d_model)
        x = self.feature_extractor(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)

        return x # (batch_size, input_seq_len, d_model)

In [266]:
sample_encoder = Encoder(num_layers=2, d_model=1280, num_heads=8, 
                         dff=256, input_vocab_size=None,
                         maximum_position_encoding=10000)

temp_input = tf.random.uniform((8, 10, 160, 160, 3), dtype=tf.float32, minval=-1, maxval=1)

sample_encoder_output = sample_encoder(temp_input, training=False, mask=None)

print (sample_encoder_output.shape) # (batch_size, input_seq_len, d_model)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: input_vocab_size is no longer used (its value of None will be ignored)
  import sys


     [get_angles] pos (10000, 1)
     [get_angles] i (1, 1280)
   [positional_encoding] angle_rads (10000, 1280)
   [FeatureExtractor.call]
   - x (8, 10, 160, 160, 3)
   - x (reshaped) (80, 160, 160, 3)
   - x (mobile_net_v2) (80, 5, 5, 1280)
   - x (global_average_layer) (80, 1280)
   - x (reshaped) (8, 10, 1280)
   [MultiHeadAttention.call]
   - v (8, 10, 1280)
   - k (8, 10, 1280)
   - q (8, 10, 1280)
     [split_heads]
     - batch_size tf.Tensor(8, shape=(), dtype=int32)
     - x (8, 10, 1280)
     - x (reshape) (8, 10, 8, 160)
     [split_heads]
     - batch_size tf.Tensor(8, shape=(), dtype=int32)
     - x (8, 10, 1280)
     - x (reshape) (8, 10, 8, 160)
     [split_heads]
     - batch_size tf.Tensor(8, shape=(), dtype=int32)
     - x (8, 10, 1280)
     - x (reshape) (8, 10, 8, 160)
     [scaled_dot_product_attention]
     - q (8, 8, 10, 160)
     - k (8, 8, 10, 160)
     - v (8, 8, 10, 160)
   - scaled_dot_product_attention (8, 8, 10, 160) (8, 8, 10, 10)
   - attention (transp

### Decoder
The `Decoder` consists of:
1. Output Embedding
2. Positional Encoding
3. N decoder layers

The target is put through an embedding which is summed with the positional encoding. The output of this summation is the input to the decoder layers. The output of the decoder is the input to the final linear layer.

In [0]:
# d_model == 1280 (comes from the encoder, i.e. images' feature vectors)
# d_model_q == 18 (comes from the previous decoder output, i.e. actions)
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
                 maximum_position_encoding, rate=0.1, d_model_q=None):
        super(Decoder, self).__init__()

        warnings.warn(f"d_model_q must be 18 as we have actions from 0 to 17 (not {d_model_q})")
        self.d_model_q = d_model_q

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model_q)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model_q)

        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate, d_model_q) for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)
    
    @indent_prints
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        print(" "*SPACES, "[Decoder.call]")
        seq_len = tf.shape(x)[1]
        attention_weights = {}

        try:
            print(" "*SPACES, "- x", x.shape)
        except:
            print(" "*SPACES, "- len(x)", len(x))
        x = self.embedding(x) # (batch_size, target_seq_len, d_model_q)
        print(" "*SPACES, "- embedding", x.shape)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]
        print(" "*SPACES, "- pos_encoding", x.shape)

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training, look_ahead_mask, padding_mask)
            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2

        # x.shape == (batch_size, target_seq_len, d_model)
        return x, attention_weights

In [268]:
sample_decoder = Decoder(num_layers=2, d_model=1280, num_heads=8, 
                         dff=256, target_vocab_size=24, 
                         maximum_position_encoding=1000, 
                         d_model_q=24)
temp_input = tf.random.uniform((8, 10), dtype=tf.int64, minval=0, maxval=18)

output, attn = sample_decoder(temp_input, 
                              enc_output=sample_encoder_output, 
                              training=False,
                              look_ahead_mask=None, 
                              padding_mask=None)

output.shape, attn['decoder_layer2_block2'].shape

     [get_angles] pos (1000, 1)
     [get_angles] i (1, 24)
   [positional_encoding] angle_rads (1000, 24)
   [Decoder.call]
   - x (8, 10)
   - embedding (8, 10, 24)
   - pos_encoding (8, 10, 24)
     [DecoderLayer.call]
     - x (8, 10, 24)
     - enc_output (8, 10, 1280)
       [MultiHeadAttention.call]
       - v (8, 10, 24)
       - k (8, 10, 24)
       - q (8, 10, 24)
         [split_heads]
         - batch_size tf.Tensor(8, shape=(), dtype=int32)
         - x (8, 10, 24)
         - x (reshape) (8, 10, 8, 3)
         [split_heads]
         - batch_size tf.Tensor(8, shape=(), dtype=int32)
         - x (8, 10, 24)
         - x (reshape) (8, 10, 8, 3)
         [split_heads]
         - batch_size tf.Tensor(8, shape=(), dtype=int32)
         - x (8, 10, 24)
         - x (reshape) (8, 10, 8, 3)
         [scaled_dot_product_attention]
         - q (8, 8, 10, 3)
         - k (8, 8, 10, 3)
         - v (8, 8, 10, 3)
       - scaled_dot_product_attention (8, 8, 10, 3) (8, 8, 10, 10)
      

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: d_model_q must be 18 as we have actions from 0 to 17 (not 24)
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: d_model_q must be 18 as we have actions from 0 to 17 (not 24)
  """


         [split_heads]
         - batch_size tf.Tensor(8, shape=(), dtype=int32)
         - x (8, 10, 24)
         - x (reshape) (8, 10, 8, 3)
         [split_heads]
         - batch_size tf.Tensor(8, shape=(), dtype=int32)
         - x (8, 10, 24)
         - x (reshape) (8, 10, 8, 3)
         [split_heads]
         - batch_size tf.Tensor(8, shape=(), dtype=int32)
         - x (8, 10, 24)
         - x (reshape) (8, 10, 8, 3)
         [scaled_dot_product_attention]
         - q (8, 8, 10, 3)
         - k (8, 8, 10, 3)
         - v (8, 8, 10, 3)
       - scaled_dot_product_attention (8, 8, 10, 3) (8, 8, 10, 10)
       - attention (transposed) (8, 10, 8, 3)
       - concat_attention (8, 10, 24)
       - output (8, 10, 24)
     - attn2 (8, 10, 24)
     - out2 (8, 10, 24)
     - out3 (8, 10, 24)
     [DecoderLayer.call]
     - x (8, 10, 24)
     - enc_output (8, 10, 1280)
       [MultiHeadAttention.call]
       - v (8, 10, 24)
       - k (8, 10, 24)
       - q (8, 10, 24)
         [split_he

(TensorShape([8, 10, 24]), TensorShape([8, 8, 10, 10]))

## Create the Transformer
Transformer consists of the encoder, decoder and a final linear layer. The output of the decoder is the input to the linear layer and its output is returned.

In [0]:
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, 
                 input_vocab_size, target_vocab_size, pe_input, pe_target, rate=0.1, 
                 d_model_q=None):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, rate)
        self.decoder = Decoder(num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, rate, d_model_q)
        self.final_layer = tf.keras.layers.Dense(target_vocab_size)
    
    def call(self, inp, tar, training, enc_padding_mask, look_ahead_mask, dec_padding_mask):
        enc_output = self.encoder(inp, training, enc_padding_mask) # (batch_size, inp_seq_len, d_model)

        # dec_output.shape == (batch_size, tar_seq_len, d_model_q)
        dec_output, attention_weights = self.decoder(tar, enc_output, training, look_ahead_mask, dec_padding_mask)

        final_output = self.final_layer(dec_output) # (batch_size, tar_seq_len, target_vocab_size)

        return final_output, attention_weights

In [270]:
sample_transformer = Transformer(
    num_layers=2, d_model=1280, num_heads=8, dff=256, 
    input_vocab_size=None, target_vocab_size=24, 
    pe_input=100, pe_target=100, # https://www.youtube.com/watch?v=HNtz05bhI1k
    d_model_q=24)

#temp_input = tf.random.uniform((64, 10, 160, 160, 3), dtype=tf.int64, minval=-1, maxval=1)
temp_input = tf.random.uniform((1, 10, 160, 160, 3), dtype=tf.float32, minval=-1, maxval=1)
temp_target = tf.random.uniform((1, 10), dtype=tf.int64, minval=0, maxval=17)

fn_out, _ = sample_transformer(temp_input, temp_target, training=False, 
                               enc_padding_mask=None, 
                               look_ahead_mask=None,
                               dec_padding_mask=None)

fn_out.shape # (batch_size, tar_seq_len, target_vocab_size)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: input_vocab_size is no longer used (its value of None will be ignored)
  import sys


     [get_angles] pos (100, 1)
     [get_angles] i (1, 1280)
   [positional_encoding] angle_rads (100, 1280)
     [get_angles] pos (100, 1)
     [get_angles] i (1, 24)
   [positional_encoding] angle_rads (100, 24)
   [FeatureExtractor.call]
   - x (1, 10, 160, 160, 3)
   - x (reshaped) (10, 160, 160, 3)
   - x (mobile_net_v2) (10, 5, 5, 1280)
   - x (global_average_layer) (10, 1280)
   - x (reshaped) (1, 10, 1280)
   [MultiHeadAttention.call]
   - v (1, 10, 1280)
   - k (1, 10, 1280)
   - q (1, 10, 1280)
     [split_heads]
     - batch_size tf.Tensor(1, shape=(), dtype=int32)
     - x (1, 10, 1280)
     - x (reshape) (1, 10, 8, 160)
     [split_heads]
     - batch_size tf.Tensor(1, shape=(), dtype=int32)
     - x (1, 10, 1280)
     - x (reshape) (1, 10, 8, 160)
     [split_heads]
     - batch_size tf.Tensor(1, shape=(), dtype=int32)
     - x (1, 10, 1280)
     - x (reshape) (1, 10, 8, 160)
     [scaled_dot_product_attention]
     - q (1, 8, 10, 160)
     - k (1, 8, 10, 160)
     - v (1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: d_model_q must be 18 as we have actions from 0 to 17 (not 24)
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: d_model_q must be 18 as we have actions from 0 to 17 (not 24)
  """


   [MultiHeadAttention.call]
   - v (1, 10, 1280)
   - k (1, 10, 1280)
   - q (1, 10, 1280)
     [split_heads]
     - batch_size tf.Tensor(1, shape=(), dtype=int32)
     - x (1, 10, 1280)
     - x (reshape) (1, 10, 8, 160)
     [split_heads]
     - batch_size tf.Tensor(1, shape=(), dtype=int32)
     - x (1, 10, 1280)
     - x (reshape) (1, 10, 8, 160)
     [split_heads]
     - batch_size tf.Tensor(1, shape=(), dtype=int32)
     - x (1, 10, 1280)
     - x (reshape) (1, 10, 8, 160)
     [scaled_dot_product_attention]
     - q (1, 8, 10, 160)
     - k (1, 8, 10, 160)
     - v (1, 8, 10, 160)
   - scaled_dot_product_attention (1, 8, 10, 160) (1, 8, 10, 10)
   - attention (transposed) (1, 10, 8, 160)
   - concat_attention (1, 10, 1280)
   - output (1, 10, 1280)
   [Decoder.call]
   - x (1, 10)
   - embedding (1, 10, 24)
   - pos_encoding (1, 10, 24)
     [DecoderLayer.call]
     - x (1, 10, 24)
     - enc_output (1, 10, 1280)
       [MultiHeadAttention.call]
       - v (1, 10, 24)
       - 

TensorShape([1, 10, 24])

## Set hyperparameters

In [0]:
num_layers = 4
d_model = 1280
d_model_q = 24
dff = 512
#num_heads = 8
num_heads = 2 # FIXME fix asserts above to use 8 instead of 2

In [0]:
N_OF_INP_IMAGES = 1000
INP_IMAGE_SHAPE = (160, 160, 3) # Atari images are 210x160 RGB.. so you should resize this somewhere mate ;)
INP_IMAGE_SIZE  = np.prod(INP_IMAGE_SHAPE)

A total of 18 actions can be performed with the joystick: doing nothing, pressing the action button, going in one of 8 directions (up, down, left and right as well as the 4 diagonals) and going in any of these directions while pressing the button.

In [0]:
input_vocab_size = None # NOTE this is no longer used
target_vocab_size = 24 # actions to take
dropout_rate = 0.1

pe_size = 100

## Optimizer
Use the Adam optimizer with a custom learning rate scheduler according to the formula in the [paper](https://arxiv.org/abs/1706.03762): ${lrate = d_{model}^{-0.5} \cdot min(step{\_}num^{-0.5},\, step{\_}num \cdot warmup{\_}steps^{-1.5})}$


In [0]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [0]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

## Loss and metrics
Since the target sequences are padded, it is important to apply a padding mask when calculating the loss.

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [0]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## Training and checkpointing


In [279]:
transformer = Transformer(num_layers, d_model, num_heads, dff,
                          input_vocab_size, target_vocab_size, 
                          pe_input=pe_size, 
                          pe_target=pe_size,
                          rate=dropout_rate,
                          d_model_q=d_model_q)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: input_vocab_size is no longer used (its value of None will be ignored)
  import sys


     [get_angles] pos (100, 1)
     [get_angles] i (1, 1280)
   [positional_encoding] angle_rads (100, 1280)
     [get_angles] pos (100, 1)
     [get_angles] i (1, 24)
   [positional_encoding] angle_rads (100, 24)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: d_model_q must be 18 as we have actions from 0 to 17 (not 24)
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: d_model_q must be 18 as we have actions from 0 to 17 (not 24)
  """


In [0]:
@indent_prints
def create_masks(inp, tar):
    print(" "*SPACES, "[create_masks]")
    # Encoder padding mask
    enc_padding_mask = create_padding_mask(inp)
    print(" "*SPACES, "- enc_padding_mask", enc_padding_mask.shape)

    # Used in the 2nd attention block in the decoder.
    # This padding mask is used to mask the encoder outputs.
    dec_padding_mask = create_padding_mask(inp)
    print(" "*SPACES, "- dec_padding_mask", dec_padding_mask.shape)

    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by 
    # the decoder.
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    print(" "*SPACES, "- look_ahead_mask", look_ahead_mask.shape)
    dec_target_padding_mask = create_padding_mask(tar)
    print(" "*SPACES, "- dec_target_padding_mask", dec_target_padding_mask.shape)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
    print(" "*SPACES, "- combined_mask", combined_mask.shape)

    return enc_padding_mask, combined_mask, dec_padding_mask

Create the checkpoint path and the checkpoint manager. This will be used to save checkpoints every `n` epochs.

In [281]:
checkpoint_path = "./checkpoints/train"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')

Latest checkpoint restored!!


### Setup input pipeline
Load data

In [0]:
import os

In [283]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [284]:
!ls drive/My\ Drive/unicamp/MC886/atari/data/

Breakout-v0_1000	      Breakout-v4_1000_actions.npy
Breakout-v0_1000_actions.npy  Pong-v0_1000
Breakout-v4_1000	      Pong-v0_1000_actions.npy


In [0]:
actions_taken = np.load('drive/My Drive/unicamp/MC886/atari/data/Breakout-v4_1000_actions.npy')
actions_taken = np.array(actions_taken, dtype='int8')

actions_data = np.array([actions_taken[:100] for _ in range(10)], dtype='int8')

In [0]:
def get_all_png(path):
    files = []
    files_full = []
    for root, dirs, file_names in os.walk(path):
        files.extend([name for name in file_names if '.png' in name.lower()])
        files_full.extend([os.path.join(root, name) for name in file_names if '.png' in name.lower()])
    return files, files_full

In [0]:
IMG_SIZE = 160 # All images will be resized to 160x160

def load_image(image_path):#, action):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image#, action
  
def load_all_imgs(paths, actions):
    print(paths[0])
    imgs = tf.map_fn(load_image, paths)
    return imgs, actions

In [288]:
_, img_name_vector = get_all_png('drive/My Drive/unicamp/MC886/atari/data/Breakout-v4_1000/')
print(len(img_name_vector))

loaded_imgs = [load_image(path) for path in img_name_vector[:100]]

img_name_all = [loaded_imgs for _ in range(10)]

1000


In [289]:
# Feel free to change batch_size according to your system configuration
image_dataset = tf.data.Dataset.from_tensor_slices((img_name_all, actions_data))

print(image_dataset.element_spec)

image_dataset = image_dataset.batch(2)
# .map(load_all_imgs, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(2)

(TensorSpec(shape=(100, 160, 160, 3), dtype=tf.float32, name=None), TensorSpec(shape=(100,), dtype=tf.int8, name=None))


In [290]:
print(actions_taken.shape)
print(len(img_name_vector))

(1000,)
1000


In [291]:
train_dataset = image_dataset

train_dataset.element_spec

(TensorSpec(shape=(None, 100, 160, 160, 3), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 100), dtype=tf.int8, name=None))

### Training

The target is divided into `tar_inp` and `tar_real`. `tar_inp` is passed as an input to the decoder. `tar_real` is that same input shifted by 1: At each location in `tar_input`, `tar_real` contains the  next token that should be predicted.

The transformer is an auto-regressive model: it makes predictions one part at a time, and uses its output so far to decide what to do next. 

During training this example uses teacher-forcing. Teacher forcing is passing the true output to the next time step regardless of what the model predicts at the current time step.

As the transformer predicts each token, *self-attention* allows it to look at the previous tokens in the input sequence to better predict the next one.

To prevent the model from peaking at the expected output the model uses a look-ahead mask.

In [0]:
EPOCHS = 20

In [0]:
# The @tf.function trace-compiles train_step into a TF graph for faster
# execution. The function specializes to the precise shape of the argument
# tensors. To avoid re-tracing due to the variable sequence lengths or variable
# batch sizes (the last batch is smaller), use input_signature to specify
# more generic shapes.

train_step_signature = [
    tf.TensorSpec(shape=(None, None, 160, 160, 3), dtype=tf.float32),
    tf.TensorSpec(shape=(None, None), dtype=tf.int8),
]

_feature_extractor = FeatureExtractor()

@indent_prints
@tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]
    print(" "*SPACES, "[train_step]")
    print(" "*SPACES, "- tar", tar.shape)
    print(" "*SPACES, "- tar_inp", tar_inp.shape)
    print(" "*SPACES, "- tar_real", tar_real.shape)

    # > enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)
    inp_feat_vec = _feature_extractor(inp)
    print(" "*SPACES, "- inp", inp.shape)
    print(" "*SPACES, "- inp_feat_vec", inp_feat_vec.shape)

    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp_feat_vec, tar_inp)

    with tf.GradientTape() as tape:
        # predictions, _ = transformer(inp, tar_inp, 
        #                              True, 
        #                              enc_padding_mask, 
        #                              combined_mask, 
        #                              dec_padding_mask)
        # FIXME some mask has a wrong shape
        predictions, _ = transformer(inp, tar_inp, 
                                     True, None, None, None)
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)
    train_accuracy(tar_real, predictions)

In [0]:
# for (batch, (inp, tar)) in enumerate(train_dataset):
#     print(inp.shape, tar.shape)
#     print(_feature_extractor(inp).shape)
#     print()
#     train_step(inp, tar)
#     break

Atari frame images are used as the input language and Atari actions are the target language.

In [295]:
for epoch in range(EPOCHS):
    start = time.time()

    train_loss.reset_states()
    train_accuracy.reset_states()

    # inp -> observations, tar -> actions
    for (batch, (inp, tar)) in enumerate(train_dataset):
        print(inp.shape, tar.shape)
        train_step(inp, tar)

        if batch % 50 == 0:
            print('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(
                  epoch + 1, batch, train_loss.result(), train_accuracy.result()))
        
    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                             ckpt_save_path))

    print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, 
                                                         train_loss.result(), 
                                                         train_accuracy.result()))

    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

(2, 100, 160, 160, 3) (2, 100)
   [train_step]
   - tar (None, None)
   - tar_inp (None, None)
   - tar_real (None, None)
     [FeatureExtractor.call]
     - x (None, None, 160, 160, 3)
     - x (reshaped) (None, None, None, None)
     - x (mobile_net_v2) (None, None, None, 1280)
     - x (global_average_layer) (None, 1280)
     - x (reshaped) (None, None, None)
   - inp (None, None, 160, 160, 3)
   - inp_feat_vec (None, None, None)
     [create_masks]
     - enc_padding_mask (None, 1, 1, None, None)
     - dec_padding_mask (None, 1, 1, None, None)
     - look_ahead_mask (None, None)
     - dec_target_padding_mask (None, 1, 1, None)
     - combined_mask (None, 1, None, None)
     [FeatureExtractor.call]
     - x (None, None, 160, 160, 3)
     - x (reshaped) (None, None, None, None)
     - x (mobile_net_v2) (None, None, None, 1280)
     - x (global_average_layer) (None, 1280)
     - x (reshaped) (None, None, None)
     [MultiHeadAttention.call]
     - v (None, None, 1280)
     - k (None

## Evaluate
TODO continue from [this section](https://www.tensorflow.org/tutorials/text/transformer#evaluate) onwards.

In [0]:
MAX_LENGTH = 100

In [0]:
@indent_prints
def evaluate(inp_sentence, last_action, max_len=None):
    print(" "*SPACES, "[evaluate]")
    print(" "*SPACES, "- inp_sentence", inp_sentence.shape)
    print(" "*SPACES, "- last_action", len(last_action))
    decoder_input = last_action
    # output = tf.expand_dims(decoder_input, 0)
    output = last_action
    print(" "*SPACES, "- output:", output)

    for i in range(MAX_LENGTH):
        # FIXME
        # enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
        #     encoder_input, output)

        # predictions.shape == (batch_size, seq_len, vocab_size)
        # predictions, attention_weights = transformer(inp_sentence, 
        #                                                 output,
        #                                                 False,
        #                                                 enc_padding_mask,
        #                                                 combined_mask,
        #                                                 dec_padding_mask)
        predictions, attention_weights = transformer(inp_sentence, decoder_input, 
                                                     False, None, None, None)
        print(" "*SPACES, "- predictions", predictions.shape)

        # select the last word from the seq_len dimension
        predictions = predictions[:, -1:, :]  # (batch_size, 1, vocab_size)
        print(" "*SPACES, "- predictions[:, -1:, :]", predictions.shape)

        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
        print(" "*SPACES, "- predicted_id", predicted_id.shape)

        # return the result if the predicted_id is equal to the end token
        # if predicted_id == tokenizer_en.vocab_size+1:
        #     return tf.squeeze(output, axis=0), attention_weights
        if max_len is not None and i > max_len:
            break

        # concatentate the predicted_id to the output which is given to the decoder
        # as its input.
        output = tf.concat([output, predicted_id], axis=-1)
        print(" "*SPACES, "- output", output.shape)

    print(" "*SPACES, "- output", output.shape)
    try:
        out = tf.squeeze(output, axis=0)
        print(" "*SPACES, "  (squeezed)")
    except:
        out = output
        print(" "*SPACES, "  (didn't squeeze)")
    print(" "*SPACES, "- out", out.shape)
    return out, attention_weights

In [0]:
SPACES = -2

In [309]:
evaluate(tf.random.uniform((2, 24, 160, 160, 3), dtype=tf.float32, minval=0, maxval=255), 
         tf.random.uniform((2, 24), dtype=tf.int32, minval=0, maxval=17),
         10)
# images: (batch_size, seq_len, 160, 160, 3)
# actions: (batch_size, seq_len)

 [evaluate]
 - inp_sentence (2, 24, 160, 160, 3)
 - last_action 2
 - output: tf.Tensor(
[[ 8  4  1 16  8  7  9  0  2  8  8  3  2 11 12  8  6 11  2 11  3 14  4  1]
 [11  9  2  3  5  7 10 11 14  1  5  3  1  4  4  9 15 12 12 16  5  7 11  8]], shape=(2, 24), dtype=int32)
   [FeatureExtractor.call]
   - x (2, 24, 160, 160, 3)
   - x (reshaped) (48, 160, 160, 3)
   - x (mobile_net_v2) (48, 5, 5, 1280)
   - x (global_average_layer) (48, 1280)
   - x (reshaped) (2, 24, 1280)
   [MultiHeadAttention.call]
   - v (2, 24, 1280)
   - k (2, 24, 1280)
   - q (2, 24, 1280)
     [split_heads]
     - batch_size tf.Tensor(2, shape=(), dtype=int32)
     - x (2, 24, 1280)
     - x (reshape) (2, 24, 2, 640)
     [split_heads]
     - batch_size tf.Tensor(2, shape=(), dtype=int32)
     - x (2, 24, 1280)
     - x (reshape) (2, 24, 2, 640)
     [split_heads]
     - batch_size tf.Tensor(2, shape=(), dtype=int32)
     - x (2, 24, 1280)
     - x (reshape) (2, 24, 2, 640)
     [scaled_dot_product_attention]
     - 

(<tf.Tensor: id=613866, shape=(2, 35), dtype=int32, numpy=
 array([[ 8,  4,  1, 16,  8,  7,  9,  0,  2,  8,  8,  3,  2, 11, 12,  8,
          6, 11,  2, 11,  3, 14,  4,  1,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  2,  2],
        [11,  9,  2,  3,  5,  7, 10, 11, 14,  1,  5,  3,  1,  4,  4,  9,
         15, 12, 12, 16,  5,  7, 11,  8,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1]], dtype=int32)>,
 {'decoder_layer1_block1': <tf.Tensor: id=615431, shape=(2, 2, 24, 24), dtype=float32, numpy=
  array([[[[0.01812618, 0.02345452, 0.00266425, ..., 0.41354075,
            0.02828045, 0.00262965],
           [0.0146041 , 0.03165805, 0.00715527, ..., 0.28450862,
            0.05005802, 0.01156814],
           [0.01483813, 0.01022757, 0.00389879, ..., 0.4778958 ,
            0.01209863, 0.00405353],
           ...,
           [0.00397873, 0.02233867, 0.05306857, ..., 0.01319798,
            0.0749886 , 0.12466578],
           [0.04303937, 0.05205029, 0.01380283, ..., 0.07389493,
      